In [1]:
# !wget -r -N -c -np -nH --cut-dirs=1 --user ntirupathirao18 --ask-password -i FILES --base=https://physionet.org/files/mimic-cxr-jpg/2.1.0/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

In [4]:
path = r'/content/drive/MyDrive/Work/llm/cxr'

In [5]:
labeled_path = r'/content/drive/MyDrive/Work/llm/cxr/labeled_reports.csv'
df = pd.read_csv(labeled_path)

In [6]:
def format_prompt(sample):
    return f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{sample["instruction"]}

### Input:
{sample["input"]}

### Response:
{sample["output"]}
"""

In [7]:
# %%capture
# %pip install accelerate peft bitsandbytes transformers trl
!pip install accelerate peft bitsandbytes transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.1/244.1 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 18.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.

In [8]:
import os
import torch
from datasets import load_dataset, Dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [9]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# New instruction dataset
guanaco_dataset = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model
new_model = "llama-2-7b-chat-guanaco"

compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [11]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_params = LoraConfig(
    # lora_alpha=16,
    # lora_dropout=0.1,
    # r=64,
    # bias="none",
    # task_type="CAUSAL_LM",

    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)




tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [12]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [13]:
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset,
#     peft_config=peft_params,
#     dataset_text_field="text",
#     max_seq_length=None,
#     tokenizer=tokenizer,
#     args=training_params,
#     packing=False,
# )

logging.set_verbosity(logging.CRITICAL)

prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
# result = pipe(f"{prompt}")

print(result[0]['generated_text'])


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<s>[INST] Who is Leonardo Da Vinci? [/INST]  Leonardo da Vinci (1452-1519) was a true Renaissance man, a polymath who excelled in various fields, including art, science, engineering, mathematics, and anatomy. everybody knows him as the most famous artist of the Italian Renaissance, but he was also a prolific inventor, engineer, and scientist. Here are some key facts about Leonardo da Vinci:

1. Early Life: Leonardo was born in Vinci, Italy, on April 15, 1452. His father, Messer Piero Fruosini, was a notary, and his mother, Caterina Buti, was a peasant.
2. Artistic Career: Leonardo began his artistic career as a young man in Florence, where he was apprenticed to the artist Andrea del Ver


In [14]:
# import os
# import shutil
# shutil.rmtree(r'/content/results')

In [15]:
df.columns

Index(['Report Impression', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices', 'No Finding'],
      dtype='object')

In [16]:
df['Report Impression'][0]

'A tip of a left Port-A-Cath lies in the low superior vena cava. Indistinct nodular opacities in the left lung base are consistent with known metastatic disease and better demonstrated on the prior CT. There is bilateral pleural thickening and small pleural effusions that appear relatively unchanged in size since ___. The cardiac and mediastinal contours are normal. The upper bowel gas pattern is unremarkable.'

In [17]:
rad_report_list = []
cancerous_findings_list = []
for indindex in range(len(df['Report Impression'])) :
  rad_report = df['Report Impression'][indindex]
  cancerous_findings = ''
  for ind_key in df.columns[1:] :
    cancerous_findings  += ind_key + ':'+ str(df[ind_key][indindex]) + ','
  rad_report_list.append(rad_report)
  cancerous_findings_list.append(cancerous_findings)


# dataset = Dataset.from_dict({'text': rad_report_list[:1000], 'label': cancerous_findings_list[:1000]})
dataset = Dataset.from_dict({'text': rad_report_list[:60000], 'label': cancerous_findings_list[:60000]})

def tokenize_data(examples):
    return tokenizer(examples['text'], text_target=examples['label'], padding='max_length', truncation=True )

tokenized_dataset = dataset.map(tokenize_data, batched=True, remove_columns=['text', 'label'])


Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [18]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [19]:
# trainer.train()
# trainer.model.save_pretrained(new_model)
# trainer.tokenizer.save_pretrained(new_model)
# # model.save_pretrained("/result/weights")
# # model.base_model.save_pretrained("/result/model", is_main_process=True, max_shard_size="2GB")

In [29]:
# cancerous_findings_temp=''
# df = df
def cancerous_findings_func(tar_index) :
  cancerous_findings_temp=''
  for ind_key in df.columns[1:] :
    cancerous_findings_temp  += ind_key + ':'+ str(df[ind_key][tar_index]) + ','
  return cancerous_findings_temp
def call_report(tar_index) :
  rad_report = df['Report Impression'][tar_index]
  cancerous_findings =  cancerous_findings_func(tar_index)
  return rad_report, cancerous_findings

rad_list = ''
for indindex in range(len(df['Report Impression'])) :
  for indshot_index in range(0,3) :
    rad_report, cancerous_findings =  call_report(indindex + indshot_index)
    rad_list += f'Radiology report {indshot_index +1 } : {rad_report} \n cancerous findings {indshot_index+1} : {cancerous_findings}\n'
  tar_rad_report, tar_cancerous_findings =  call_report(indindex + indshot_index)


  # print(rad_report)
  # print(cancerous_findings)


  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000)
  result = pipe(f'''
  please find the radiology report on the scan , try to extract individual findings are present or absent from the radiology report

>In cancerous findings 1.0 represents present , 0.0 represent absent , nan represent there is no information about it
consider below an examples of radiology report and cancerous findings
{rad_list}

please find the cancerous findings for this new radiology report
tar Radiology report  : {tar_rad_report}

  ''')
  # result = pipe(f'''{prompt}''')
  # result = pipe(f"<s>[INST] {rad_report} [/INST]")
  # print(prompt)

  print(result[0]['generated_text'].strip())
  print('---------------------------------------------------------------')
  print('   cancerous findings  : ', tar_cancerous_findings)
  print(result[0]['generated_text'] == tar_cancerous_findings)
  if indindex == 1 :
    break

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


please find the radiology report on the scan , try to extract individual findings are present or absent from the radiology report

>In cancerous findings 1.0 represents present , 0.0 represent absent , nan represent there is no information about it
consider below an examples of radiology report and cancerous findings
Radiology report 1 : A tip of a left Port-A-Cath lies in the low superior vena cava. Indistinct nodular opacities in the left lung base are consistent with known metastatic disease and better demonstrated on the prior CT. There is bilateral pleural thickening and small pleural effusions that appear relatively unchanged in size since ___. The cardiac and mediastinal contours are normal. The upper bowel gas pattern is unremarkable. 
 cancerous findings 1 : Enlarged Cardiomediastinum:0.0,Cardiomegaly:0.0,Lung Opacity:nan,Lung Lesion:1.0,Edema:nan,Consolidation:nan,Pneumonia:nan,Atelectasis:nan,Pneumothorax:nan,Pleural Effusion:1.0,Pleural Other:1.0,Fracture:nan,Support Device

In [ ]:

prompt =df['Report Impression'][0]
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer , max_length = 100)
# result = pipe(f"<s>[INST] {prompt} [/INST]")
result = pipe(f'''{prompt}''')
print(prompt)

print(result[0]['generated_text'])

In [ ]:
logging.set_verbosity(logging.CRITICAL)

input_text = rad_report_list[1002]
# from transformers import LlamaForCausalLM
# base_model = LlamaForCausalLM.from_pretrained("path/to/save/base/model", load_in_8bit=True, device_map="auto")

input_ids = trainer.tokenizer.encode(input_text, return_tensors="pt")
output_ids = trainer.model.generate(input_ids, max_length=1024, do_sample=True, top_k=50, top_p=0.95, num_return_sequences=1)
output_text = trainer.tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)
print(len(output_text))
print(input_text)